# Load h5

In [1]:
from tensorflow import keras

model = keras.models.load_model("./trained_models/kws_model.h5")

2024-12-12 16:58:13.488499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733993893.505612  452176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733993893.510592  452176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 16:58:13.527422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1733993895.550779  452176 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0

In [2]:
model.input

<KerasTensor shape=(None, 49, 10, 1), dtype=float32, sparse=False, name=input_layer>

In [8]:
import numpy as np

# 定義輸入形狀 (49, 10, 1)
input_shape = (5,49, 10, 1)

# 生成隨機輸入數據，值域在 0 和 1 之間（可根據需要調整範圍）
random_data = np.random.random(input_shape).astype(np.float32)

# 檢查生成數據的形狀
print("Random input data shape:", random_data.shape)

Random input data shape: (5, 49, 10, 1)


In [12]:
model(random_data)

<tf.Tensor: shape=(5, 12), dtype=float32, numpy=
array([[3.3107188e-02, 1.8307546e-02, 2.2391740e-02, 9.8935943e-03,
        3.5346135e-02, 4.8190063e-01, 1.2375110e-02, 4.7549251e-03,
        2.0070558e-02, 1.1276059e-02, 1.9837824e-04, 3.5037810e-01],
       [3.3474151e-02, 1.8397849e-02, 2.2186341e-02, 1.1371152e-02,
        2.8211489e-02, 4.6600327e-01, 1.2240857e-02, 5.2732271e-03,
        1.7311089e-02, 1.1821508e-02, 1.7622387e-04, 3.7353286e-01],
       [2.8724747e-02, 1.4749967e-02, 2.3112694e-02, 7.8635011e-03,
        3.6105108e-02, 4.7681385e-01, 1.2290912e-02, 5.0713215e-03,
        1.9153949e-02, 1.0088203e-02, 1.6789736e-04, 3.6585784e-01],
       [3.0150685e-02, 1.6742840e-02, 2.6668383e-02, 9.3297921e-03,
        3.1946670e-02, 4.5188710e-01, 1.4934554e-02, 5.5071409e-03,
        1.6808454e-02, 1.0293491e-02, 1.7482247e-04, 3.8555601e-01],
       [3.8017865e-02, 1.7025813e-02, 2.3796320e-02, 1.0138495e-02,
        2.8503824e-02, 4.5935825e-01, 1.2409093e-02, 6.1739879e

# Load tflite

In [2]:
import numpy as np
import tensorflow as tf

# 1. 載入 TensorFlow Lite 模型
interpreter = tf.lite.Interpreter(model_path="./trained_models/kws_model_float32.tflite")

# 2. 配置解譯器
interpreter.allocate_tensors()

# 3. 檢查輸入/輸出細節
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([ 1, 49, 10,  1], dtype=int32), 'shape_signature': array([-1, 49, 10,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 34, 'shape': array([ 1, 12], dtype=int32), 'shape_signature': array([-1, 12], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
# 4. 生成隨機輸入數據
input_shape = input_details[0]['shape']  # 獲取輸入形狀
random_input = np.random.random(input_shape).astype(np.float32)

# 5. 將輸入數據設置到解譯器
interpreter.set_tensor(input_details[0]['index'], random_input)

# 6. 執行推論
interpreter.invoke()

# 7. 獲取輸出結果
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Model output:", output_data)

Model output: [[3.1352654e-02 1.5878821e-02 2.4776682e-02 9.4797174e-03 3.2707583e-02
  4.6977413e-01 1.3405719e-02 5.6135906e-03 1.9217107e-02 1.0959848e-02
  1.7620620e-04 3.6665797e-01]]
